In [1]:
import db_conn
import pandas as pd
import re

/home/grace/.local/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
conn = db_conn.get_connection()

## 1. 전체 문장에서 약물 추출하기

In [3]:
def get_total_drugs():
    cur = conn.cursor()
#     cur.execute("""
#     SELECT DISTINCT(TRIM(d.tt_drug)) AS t_d
#     FROM
#     (SELECT id, tt_drug
#     FROM article_table_sentences_m M,
#         UNNEST(STRING_TO_ARRAY(t_drug, ' , ')) AS tt_drug
#     WHERE (tt_drug='') is false) as d;
#     """)

# topic num 지정됨
    cur.execute("""
    SELECT id, TRIM(lower(d.tt_drug)) AS t_d
    FROM
    (SELECT id, tt_drug
    FROM 
        (SELECT * 
        FROM article_table_sentences_m
        WHERE table_id IN 
        (SELECT id FROM article_tables WHERE final_topic IS NOT NULL)) M,
        UNNEST(STRING_TO_ARRAY(t_drug, ' , ')) AS tt_drug
    WHERE (tt_drug='') is false) as d;
    """)
    return cur.fetchall()

def get_drug_dictionary():
    cur = conn.cursor()
    cur.execute("""
    select cui_id, trim(lower(val)) from (
    (select cui1 as cui_id, cui1_str as val from dict_collapsed_final where cui1 is not null)
    UNION
    (select cui2 as cui_id, cui2_str as val from dict_collapsed_final where cui2 is not null)
    ) final;
    """)
    return cur.fetchall()

def get_drug_raw():
    cur = conn.cursor()
    cur.execute("""
    select cui1, trim(lower(str)) from
    drug_180622 where cui1 is not null;
    """)
    return cur.fetchall()

In [4]:
drugs = pd.DataFrame(get_total_drugs())

In [5]:
# drugs.to_csv('./drugs.csv', sep=',')

In [6]:
drugs.columns=['sent_id', 'name']

In [7]:
drugs['refined'] = ''
drugs['cui_id'] = ''
drugs.head()

,sent_id,name,refined,cui_id
0,2486730,ramipril,,
1,2486730,telmisartan,,
2,2602695,letrozole,,
3,2602695,palbociclib,,
4,1318105,efavirenz,,


#### 전처리 전 총 약물의 수는 137963개 입니다

In [8]:
drugs.shape

(137963, 4)

### 2. 전처리

A. dictionary를 참고하여, 정확히 일치하는 약물명은 전처리 없이 포함합니다.

In [9]:
dictionary = pd.DataFrame(get_drug_dictionary())
dictionary.columns = ['cui_id', 't_drug']
# dictionary.drop_duplicates(inplace=True)
dictionary.shape

(23931, 2)

In [10]:
raw_dictionary = pd.DataFrame(get_drug_raw())
raw_dictionary.columns = ['cui_id', 't_drug']
# raw_dictionary.drop_duplicates(inplace=True)
raw_dictionary.shape

(12291, 2)

In [11]:
# intersected_df = pd.merge(dictionary, raw_dictionary, how='inner')
# intersected_df
dictionary = pd.concat([raw_dictionary, dictionary], join='outer')
dictionary = dictionary[dictionary.cui_id!='']
dictionary.drop_duplicates(subset='t_drug', keep='first', inplace=True)
dictionary.reset_index(drop=True)
dictionary.t_drug = dictionary.t_drug.apply(lambda x: x.strip().lower())

In [12]:
len(dictionary)

25934

In [13]:
dictionary[dictionary.t_drug=='albumin']

,cui_id,t_drug
177,C0001924,albumin


In [14]:
def apply_dictionary(_drugs):
    for ind, dr in _drugs[_drugs['refined'] == ''].iterrows():
    #     print(ad['name'].strip())
        name = dr['name'].strip().lower()
        exact_mathing = dictionary.loc[dictionary.t_drug==name]

        if len(exact_mathing)==0:
            if name[0] in ['"', '.', ':', '-', '(', ')']:
                name = name[1:]
            if name[len(name)-1] in ['"', '.', ':', '-', '(', ')']:
                name = name[:len(name)-1]
    #         print(name)
            exact_mathing = dictionary.loc[dictionary.t_drug==name]

        if len(exact_mathing)>0:
#             print(exact_mathing.values)
            _drugs.loc[ind, 'refined'] = name
            _drugs.loc[ind, 'cui_id'] = exact_mathing.values[0][0]
#             print(_drugs.loc[ind])
    return _drugs

drugs = apply_dictionary(drugs)

In [15]:
drugs[drugs.refined!= ''].shape

(130363, 4)

In [16]:
drugs[drugs.refined== ''].shape

(7600, 4)

#### B. 약물을 포함하고 있는 구절이 추출 된 경우, 약물 외적인 단어들을 제거합니다

## 선생님께서 need_to_refined_hyj.xlsx파일을 통해 약물 이름이 아니라고 지정해주신 목록을 추가하였습니다.

In [17]:
replace_with_space = ['-long-acting', '-treated', '-producing', '-high', '-involved-field', '-based', '-studies', '-non-susceptible',\
                      '-susceptible', '-dependent', '-inducible', '-independent', '-eligible', '-rescue', '-sensitive', '-dense',\
                      'infarction-', '11c]-', 'log[', '[99mtc]', '99mtc', '-avid', '-subcutaneous', '-coated', '-activatable', '-total', '-corrected', \
                      '-predicted', '-gated', '-redo-surgery', '-group', '-assigned', '-allocated', '-reduced', '-combination', \
                      '-standard', '-associated', '-ischemic', '-resistant', '-naive', '-sparing', 'concentration', '[', ']',\
                      'low-molecular-weight-', '-extended', '-meter', '-stimulated', 'subtotal', '-labeled', '-matching',\
                      '-resistance;', '-maintenance', '-rate', '-facilitated', '-replacement', '-to-renin', '-gits', '-treated',\
                      '-right', '-or-2-h', '-treatment', '-boosting', '-resistance', '-continuation', '-rich', '&#8211',\
                      '-impregnated', '-dispensation', '-pemetrexed', '-correctable', '-carry-on', '-oral', '-intravenous', '-bare-metal', '-dose',\
                      '-equivalent', '-induced', '-positive', '-related', '-stimulating', '-confirmed', '-controlled', '(', '-level',\
                      '-releasing', '-reuptake', '-eluting', '-containing', '-binding', 'vaccines', 'vaccine', 'change', 'vsstandard',\
                      '-sham', '-adjusted', 'post-', '5m-', '20m-', '-provoked', '<3.2', '<3.3', '30-', '-synthase', '-responsive',\
                      '-pa-824', 'formulation-', 'mi-', 'discretion-', 'polychemotherapy', 'chemotherapy', 'seizures-','crack-',\
                      '-myocardial', 'exercise-', '—4803', '-low', 'mothers', 'von willebrand factor', 'others', '—1688', '-13',
                      # 약물 이름이 아닌 경우
                      'food-borne', 'fast-food', 'food-coping', 'grass-pea-food', 'food-systems', 'food-bolus', 'food-specific', \
                      'food-frequency', 'non-food', 'blood plasma', 'medical air', 'tree pollen', 'grass pollen', 'non-naproxen',\
                      'prothrombin', 'hypoalbuminaemia', 'multifood', 'epithelium', 'microalbuminuria', 'eosinophils',
                      'non-bevacizumab', 'albumin-creatinine', 'foods', 'non-cetuximab', 'eosinophilia', 'eosinophil', 'nonprotamine',
                      'albuminuria', ' :', 'protein', 'potassium', 'pyronaridine artesunate', 'pyronaridine', 'azt',
                      # 추가 약물이름이 아닌 경우
                      'post-food', 'tc-sulfur', 'iodine-to-creatinine', 'protein c2', 'tc–sulfur', 'blood-glucose', \
                      'cy5.5-arginine-glycine-aspartate', 'biotin-aacctgaccacagaggagaggc-3', 'n-2-mercaptopropionylglycine', \
                      'biotin-tccctcatccctgccatgt-3', 'animals-l-arg', 'biotin-cacggcacctggcctaaag-3', 'buserelin+cpax14',\
                      # 추가 확인 필요
                      'albumin:creatinine', 'glucose-insulin-potassium', 'oxygen-18', 'cyclooxygenase-2', '18f-fluorodeoxyglucose', 'cyclo-oxygenase'
                     ]

In [18]:
rep = dict((re.escape(k), ' ') for k in replace_with_space)
pattern = re.compile("|".join(rep.keys()))

for ind, row in drugs.iterrows():
    if row['refined']=='':
        refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
        if refined.strip() == '':
            drugs.drop(ind, inplace=True)
        elif row['name'].strip() != refined.strip():
            drugs.loc[ind, 'refined'] = refined.strip()
            print(row['name'])
            print(drugs.loc[ind, 'refined'])
        else:
            print(row['name'])
        print('===============================')
        refined = None

drugs.reset_index(drop=True)

epirubicin-cyclophosphamide
epirubicin-cyclophosphamide
epirubicin-cyclophosphamide
paclitaxel-eluting
paclitaxel
paclitaxel-eluting
paclitaxel
paclitaxel-eluting
paclitaxel
paclitaxel-eluting
paclitaxel
epirubicin-cyclophosphamide
epirubicin-cyclophosphamide
paclitaxel-gemcitabine
epirubicin-cyclophosphamide
paclitaxel-gemcitabine
epirubicin-cyclophosphamide
fluorouracil-t
ceftolozane-tazobactam
ceftolozane-tazobactam
ceftolozane-tazobactam
irinotecan–temozolomide–dinutuximab
irinotecan–temozolomide–dinutuximab
irinotecan–temozolomide–temsirolimus
irinotecan–temozolomide–temsirolimus
irinotecan–temozolomide–dinutuximab
dihydroartemisinin–piperaquine
dihydroartemisinin–piperaquine
tauroursodeoxycholic acid
warfarin–enoxaparin
quetiapine-er
paclitaxel-eluting
paclitaxel
paclitaxel-eluting
paclitaxel
paclitaxel-eluting
paclitaxel
mgbendroflumethiazide
mgbendroflumethiazide
mgbendroflumethiazide
mgpotassiumdoxazosin
mg doxazosin
peginterferon
bortezomibrituximab
capecitabine-cisplatin
cap

rituximab-dose-dense
rituximab
peg-interferon-alfa-2a
dolutegravir-rilpivirine
dolutegravir-rilpivirine
dolutegravir-rilpivirine
atovaquone-proguanil
salmeterolplacebo
salmeterolroflumilast
roflumilast-treated
roflumilast
roflumilast-treated
roflumilast
roflumilast-treated
roflumilast
abacavir–lamivudine
atovaquone-proguanil
budesonide-formoterol
chloroquine-proguanil
rituximab-dose-dense
rituximab
dobutamine-atropine
dipyridamole-atropine
no-irinotecan
ganciclovir-treated
ganciclovir
albumin-to-creatinine
albumin-to-creatinine
everolimus-eluting
everolimus
lopinavir-ritonavir
chemotherapycetuximab
cetuximab
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
everolimus-eluting
everolimus
dihydroartemisinin–piperaquine
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
peginterferon
pre-carvedilol


glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
glucose-corrected
glucose
dolutegravir

lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  chol

albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to-creatinine
albumin-to

...

adalimumab-rescue
adalimumab
adalimumab-carry-on
adalimumab
methotrexate-monotherapy
methotrexate-monotherapy
ceftriaxone-resistant
ceftriaxone
etoposideifosfamidedoxorubicin
etoposideifosfamidedoxorubicin
etoposideifosfamidedoxorubicin
etoposideifosfamidedoxorubicin
etoposideifosfamidedoxorubicin
etoposideifosfamidedoxorubicin
fluorouracil-based
fluorouracil
fluorouracil-based
fluorouracil
capecitabine-based
capecitabine
capecitabine-based
capecitabine
fluorouracil-based
fluorouracil
fluorouracil-based
fluorouracil
capecitabine-based
capecitabine
capecitabine-based
capecitabine
fluorouracil-based
fluorouracil
nicotine-free
capecitabine-based
capecitabine
artesunate–amodiaquine
dihydroartemisinin–piperaquine
artemether–lumefantrine
artesunate–amodiaquine
dihydroartemisinin–piperaquine
pyronaridine–artesunate
–artesunate
artemether–lumefantrine
artemether–lumefantrine
pyronaridine–artesunate
–artesunate
pyronaridine–artesunate
–artesunate
pyronaridine–artesunate
–artesunate
artesunate–a

chemotherapycetuximab
cetuximab
chemotherapycetuximab
cetuximab
chemotherapycetuximab
cetuximab
capecitabine-cisplatin
capecitabine-cisplatin
capecitabine-cisplatin
capecitabine-cisplatin
capecitabine-cisplatin
capecitabine-cisplatin
capecitabine-cisplatin
capecitabine-cisplatin
capecitabine-cisplatin
capecitabine-cisplatin
capecitabine-cisplatin
capecitabine-cisplatin
capecitabine-cisplatin
bortezomib-rituximab
fluorouracil-plus-oxaliplatin
fluorouracil-plus-oxaliplatin
bortezomib-rituximab
bortezomib-rituximab
bortezomib-rituximab
progesterone-receptor
peg-interferon-alfa-2a
peg-interferon-alfa-2a
peg-interferon-alfa-2a
peg-interferon-alfa-2a
placebodocetaxel
vandetanibdocetaxel
placebodocetaxel
placebodocetaxel
vandetanibdocetaxel
placebodocetaxel
placebodocetaxel
vandetanibdocetaxel
vandetanibdocetaxel
placebodocetaxel
vandetanibdocetaxel
retinol-binding
retinol
paclitaxel-eluting
paclitaxel
sirolimus-eluting
sirolimus
sirolimus-eluting
sirolimus
sirolimus-eluting
sirolimus
paclita

lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
tenecteplase-alone
tenecteplase-facilitated
tenecteplase
asparaginase-associated
asparaginase
lopinavir-ritonavir
paclitaxel-eluting
paclitaxel
progesterone-receptor
paclitaxel-eluting
paclitaxel
ximelagatran-warfarin
lopinavir-ritonavir
bivalirudingpi
paclitaxel-eluting
paclitaxel
sirolimus-eluting
sirolimus
paclitaxel-eluting
paclitaxel
sirolimus-eluting
sirolimus
sirolimus-eluting
sirolimus
sirolimus-eluting
sirolimus
sirolimus-eluting
sirolimus
sirolimus-eluting
sirolimus
sirolimus-eluting
sirolimus
sirolimus-eluting
sirolimus
paclitaxel-eluting
paclitaxel
sirolimus-eluting
sirolimus
darunavir-ritonavir
atenolol-based
atenolol
sirolimus-eluting
sirolimus
sirolimus-eluting
sirolimus
ceftolozane-tazobactam
ceftolozane-tazobactam
etoposideifosfamidedoxorubicin
peginterferon
levodopa-carbidopa
levodopa-carbidopa
pre-carvedilol
paclitaxel-eluting
paclitaxel
paclitaxel-eluting
paclitaxel
sirolimus-eluting

lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol
lipoprotein cholesterol
lipo  cholesterol


,sent_id,name,refined,cui_id
0,2486730,ramipril,ramipril,C0072973
1,2486730,telmisartan,telmisartan,C0248719
2,2602695,letrozole,letrozole,C0246421
3,2602695,palbociclib,palbociclib,C3853822
4,1318105,efavirenz,efavirenz,C0674427
5,1318105,rilpivirine,rilpivirine,C1566826
6,1894944,efavirenz,efavirenz,C0674427
7,1894944,rilpivirine,rilpivirine,C1566826
8,374231,ribavirin,ribavirin,C0035525
9,374231,ritonavir,ritonavir,C0292818


#### 전처리와 약물이 아닌 레코드를 제거하면 137518개가 남습니다

In [19]:
drugs.shape

(137534, 4)

## 복합 약물 정제

### 선생님께서 주신 엑셀 파일을 기반으로 정제합니다

In [20]:
replace_excel = pd.read_excel('./need_to_refined_hyj.xlsx', header=None)
replace_excel.columns = ['id', 'key', 'replace_with', '-', '-', '-']
replace_excel = replace_excel[['key', 'replace_with']]
replace_excel.drop_duplicates(inplace=True)
replace_excel.apply(lambda x: x.astype(str).str.lower())
replace_excel.head()

,key,replace_with
0,17-beta-estradiol,17-beta-estradiol
1,17-beta-estradiol-sham,17-beta-estradiol
2,17-hydroxyprogesterone,17-hydroxyprogesterone
3,5-flucytosine,5-flucytosine
4,5-hydroxyindoleacetic acid,5-hydroxyindoleacetic acid


In [21]:
replace_excel.tail()

,key,replace_with
515,sirolimus-eluting,sirolimus
516,ec-cyclophosphamide,epirubicin + cyclophosphamide
517,tetracyclines,tetracycline
518,bp-sirolimus-eluting,sirolimus
519,radioiodine-labeled,iodine


In [22]:
replace_excel.shape

(509, 2)

In [23]:
def replace_with_manual_curation(drugs):
    for ind, row in replace_excel.iterrows():
#         if len(drugs.loc[drugs.name==row['key']])>0:
#             print(row['key'])
#             print('==================')
        for d_ind, d_row in drugs.loc[drugs.name==row['key']].iterrows():
            origin_str = d_row['name'].replace(row['key'], row['replace_with'])
            drugs.loc[d_ind, 'refined'] = origin_str 
#             print(d_row)
    return drugs

In [24]:
drugs = replace_with_manual_curation(drugs)

In [25]:
drugs[drugs.name.str.find('carvedilolmetoprolol')>=0][:2]

,sent_id,name,refined,cui_id
17118,1421397,carvedilolmetoprolol,carvedilol + metoprolol,
17120,1421404,carvedilolmetoprolol,carvedilol + metoprolol,


In [26]:
drugs[drugs.refined.str.find('zidovudine lamivudine amprenavir')>=0]

,sent_id,name,refined,cui_id


## 아직 추가 전처리가 필요한 약물 수

In [27]:
drugs[drugs.refined==''].shape

(155, 4)

#### B. 약-부작용 관계를 나타는 것이 아닌 경우, 지웁니다

### 선생님께서 피드백 주신데로 non은 제외하였습니다.
### 선생님께서 피드백 주신데로 약물명 전처리에서 collagen, blood, glucose, oxygen, nitrogen-13 등을 추가하였습니다.

In [28]:
non_drug_ad_rel = ['pseudo', 'placebo', 'negative', 'free', 'withdrawal', 'blocking', 'deficient', 'inhibiting',
                  '-prev', '-insensitive', 'resistant', 'refractory', 'pre-', 'lipoprotein', '-untreated', 'anti-',
                  '-receptor', '-tolerance', '-intolerant', 'non-secretin', 'collagen', 'blood', 'glucose', 'oxygen',\
                  'nitrogen-13', 'thrombin-antithrombin', 'fetoprotein', 'renin', 'cholesterol', 'albumin:creatinine',\
                  'non-indinavir', 'eosinophilia', 'potassium']

rep = dict((re.escape(k), ' ') for k in non_drug_ad_rel)
pattern = re.compile("|".join(rep.keys()))

In [29]:
for ind, row in drugs.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if row['name'].strip() != refined.strip():
#         print(row['name'])
        drugs.drop(ind, inplace=True)
#         print('===============================')

drugs.reset_index(drop=True)

,sent_id,name,refined,cui_id
0,2486730,ramipril,ramipril,C0072973
1,2486730,telmisartan,telmisartan,C0248719
2,2602695,letrozole,letrozole,C0246421
3,2602695,palbociclib,palbociclib,C3853822
4,1318105,efavirenz,efavirenz,C0674427
5,1318105,rilpivirine,rilpivirine,C1566826
6,1894944,efavirenz,efavirenz,C0674427
7,1894944,rilpivirine,rilpivirine,C1566826
8,374231,ribavirin,ribavirin,C0035525
9,374231,ritonavir,ritonavir,C0292818


#### 약-부작용 관계를 나타는 것이 아닌 레코드를 제거하면 133617남습니다

In [30]:
drugs.shape

(133615, 4)

### 여러 약물이 하나의 레코드에 담겨있는 경우, 분리하여 입력

In [31]:
for ind, dr in drugs[(drugs.name.str.find('-')>=0) & (drugs['refined'] == '')].iterrows():
    if dr['name'].find('-to-')>=0:
#         dr['refined'] = dr['name'].replace('-to-', '+')
        drugs.loc[ind, 'refined'] = dr['name'].replace('-to-', ' + ')

In [32]:
drugs[drugs.name.str.find('+')>=0]

,sent_id,name,refined,cui_id
12947,492349,azt+3tc+abacavir+nfv,zidovudine + lamivudine + abacavir + nelfinavir,
12949,492353,azt+3tc+abacavir+efv,zidovudine + lamivudine + abacavir + efavirenz,
33765,2039780,amodiaquine+sp,sulfadoxine + pyrimethamine + amodiaquine,
33770,2039786,amodiaquine+sp,sulfadoxine + pyrimethamine + amodiaquine,
39945,177028,tiotropium+roflumilast,tiotropium + roflumilast,
59418,177022,tiotropium+roflumilast,tiotropium + roflumilast,
59449,2039798,amodiaquine+sp,sulfadoxine + pyrimethamine + amodiaquine,
60418,2039810,amodiaquine+sp,sulfadoxine + pyrimethamine + amodiaquine,
64736,177016,tiotropium+roflumilast,tiotropium + roflumilast,
69619,2039816,amodiaquine+sp,sulfadoxine + pyrimethamine + amodiaquine,


In [33]:
def separate_unique_drugs(drugs):
    additional_drugs = []

    for ind, row in drugs[drugs.refined.str.find('+')>=0].iterrows():
        processed = list(map(lambda x: x.strip(), row['refined'].split('+')))

        if len(processed)>0:
            drugs.loc[ind, 'refined'] = processed[0]
            for pro in processed[1:]:
                additional_drugs.append({'sent_id':row['sent_id'], 'name':row['name'], 'refined':pro})

    for ind, row in drugs[drugs.refined.str.find(',')>=0].iterrows():
        processed = list(map(lambda x: x.strip(), row['refined'].split(',')))

        if len(processed)>0:
            drugs.loc[ind, 'refined'] = processed[0]
            for pro in processed[1:]:

                additional_drugs.append({'sent_id':row['sent_id'], 'name':row['name'], 'refined':pro})


    drugs = drugs.append(pd.DataFrame(additional_drugs))
    drugs.drop_duplicates(inplace=True)
    drugs.reset_index(drop=True)
    return drugs

drugs = separate_unique_drugs(drugs)

/home/grace/.local/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


#### 식별 가능한 약물명을 나누고 나면 136661개가 됩니다.

In [34]:
drugs.shape

(136754, 4)

In [35]:
drugs[drugs.name.str.find('taxane-cyclophosphamide')>=0]

,cui_id,name,refined,sent_id
1252,,taxane-cyclophosphamide,paclitaxel,2456322
39796,,mono-taxane-cyclophosphamide,paclitaxel,2456320
39,NaN,taxane-cyclophosphamide,cyclophosphamide,2456322
858,NaN,mono-taxane-cyclophosphamide,cyclophosphamide,2456320


### 정제후 다시 한번 dictionary와 매칭

In [36]:
def apply_dictionary_to_empty_cui_id(_drugs):
    for ind, dr in _drugs[_drugs['cui_id'] == ''].iterrows():
    
        if dr['refined'].strip():
            name = dr['refined'].strip().lower()
        else:
            name = dr['name'].strip().lower()
            
        exact_mathing = dictionary.loc[dictionary.t_drug==name]

        if len(exact_mathing)>0:
#             print(exact_mathing.values)
            _drugs.loc[ind, 'refined'] = name
            _drugs.loc[ind, 'cui_id'] = exact_mathing.values[0][0]
#             print(_drugs.loc[ind])
    return _drugs

drugs = apply_dictionary_to_empty_cui_id(drugs)

In [37]:
drugs[drugs.cui_id==''].shape

(72, 4)

### 앞 뒤의 -, . 제거

In [38]:
remove_list = ['.', '-', ':', '+', '–', ';', '%']
def preprocess(name):
    if name[0] in remove_list:
        name = name[1:len(name)]
    
    if name[len(name)-1] in remove_list:
        name = name[0:len(name)-1]
    return name.strip()
    
drugs.name = drugs.name.map(lambda x: preprocess(x))

In [39]:
def replace_with_manual_curation_empty_refined(_drugs):
    for ind, row in replace_excel.iterrows():
        for d_ind, d_row in _drugs.loc[(_drugs.refined=='') & (_drugs.name==row['key'])].iterrows():
            origin_str = d_row['name'].replace(row['key'], row['replace_with'])
            _drugs.loc[d_ind, 'refined'] = origin_str 
#             print(d_row)
    return _drugs
drugs = replace_with_manual_curation_empty_refined(drugs)

In [40]:
# drugs.loc[drugs.name=='digoxin']
# drugs.loc[drugs.refined=='epirubicin + cyclophosphamide']

In [41]:
drugs = separate_unique_drugs(drugs)

/home/grace/.local/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [42]:
drugs[(drugs['refined'] == '')]

,cui_id,name,refined,sent_id
4820,,acetylcholinesterase,,468608
102935,,acetylcholinesterase,,468606
107959,,acetylcholinesterase,,468605
133984,,acetylcholinesterase,,468607


In [47]:
# drugs.refined = drugs[(drugs['cui_id'] == '')].refined.map(lambda x: preprocess(x))
for ind, row in drugs[(drugs['cui_id'] == '')].iterrows():
    if row['refined'].strip() != row['refined']:
        drugs.loc[ind, 'refined'] = row['refined'].strip()
    if row['refined'].strip() == '' or row['refined'].strip() == 's':
            drugs.drop(ind, inplace=True)
    if row['refined'].strip() == 'pegloticase :':
        drugs.loc[ind, 'refined'] = 'pegloticase'

In [48]:
drugs = apply_dictionary_to_empty_cui_id(drugs)
drugs[(drugs['cui_id'] == '')].shape

(63, 4)

In [49]:
drugs[(drugs['cui_id'] == '')]['refined'].unique()

array(['cy55-arginine-glycine-aspartate', 'pyronaridine', 'folfiri'], dtype=object)

In [50]:
drugs[(drugs['cui_id'] == '')]['name'].unique()

array(['cy55-arginine-glycine-aspartate', 'pyronaridine-artesunate',
       'pyronaridine–artesunate', 'folfiri–cetuximab'], dtype=object)

In [57]:
# drugs.groupby(drugs.columns.tolist(),as_index=False).size()

## 정제된 목록입니다.

In [62]:
origin_drugs = drugs[(drugs['cui_id'] != '')]
origin_drugs.shape

(136018, 4)

In [66]:
origin_drugs.head()

,cui_id,name,refined,sent_id
0,C0072973,ramipril,ramipril,2486730
1,C0248719,telmisartan,telmisartan,2486730
2,C0246421,letrozole,letrozole,2602695
3,C3853822,palbociclib,palbociclib,2602695
4,C0674427,efavirenz,efavirenz,1318105


In [ ]:
# for ind, row in replace_excel.iterrows():
#     for d_ind, d_row in drugs.loc[(drugs['cui_id']=='') & (drugs['refined']==row['key'])].iterrows():
#         origin_str = d_row['name'].replace(row['key'], row['replace_with'])
#         print(origin_str)
#         drugs.loc[d_ind, 'refined'] = origin_str 
#         print(d_row)
# drugs = separate_unique_drugs(drugs)
# dr

In [ ]:
# drugs[drugs['refined'] == ''].to_csv('./need_to_refined_0225.csv', sep=',')

## Bulk Insert

In [65]:
import os
from io import StringIO

In [67]:
sio = StringIO()
sio.write(origin_drugs.to_csv(index=None, header=None))
sio.seek(0)

with conn.cursor() as c:
    c.copy_from(sio, "table_extracted_drug", columns = drugs.columns, sep=',')
    conn.commit()